# レポート課題2

## 実行環境

- CPU : AMD Ryzen 7 PRO 5850U
- ENV : Jupyter Notebook on Ubuntu(WSL2)

In [2]:
import numpy as np
import matplotlib.pyplot as plt

import warnings
import time

from sklearn.datasets import make_classification

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import GridSearchCV, cross_val_score, train_test_split

warnings.simplefilter('ignore')


## データセット作成

- `n_features`
	- 特徴量(次元)の数
- `n_redundant`
	- 冗長特徴量(次元)の数
- `n_informative`
	- クラスタを配置する超立方体の次元の数
- `n_classes`
	- クラスの数
- `class_sep`
	- クラスタを配置する超立方体の大きさ(比率)

`class_sep`を0.5程度とし，ベースライン手法(後述のk-NN法)で20%以上の認識誤りを発生するデータとしている．

In [4]:
""" samples """

X, y = make_classification(
    n_features=4,
    n_redundant=0,
    n_informative=4,
    n_classes=2,
    class_sep=0.5,
)

# plt.scatter(X[:, 0], X[:, 1], marker='o', c=y)


## パターン認識

### パラメータ
- kNN(k近傍法)
	- n_neighbors: kの値 
- SVM(サポートベクタマシン，RBFカーネル)
	- C: 正則化に用いる係数
	- kernel: カーネル関数
- MLP(ニューラルネットワーク，多層パーセプトロン)
	- alpha: L2正則化の強さ
- NB(単純ベイズ法)
	- var_smoothing: 分布を平滑化するために分布に追加するサンプル

In [5]:
""" classifires """

names = [
    'kNN',
    'SVM',
    'MLP',
    'NB',
]

clfs = [
    KNeighborsClassifier(),
    SVC(kernel='rbf'),
    MLPClassifier(activation='relu', max_iter=1000),
    GaussianNB(),
]

grid_params = [
    {
        'n_neighbors': np.arange(1, 11),
    },
    {
        'C': [0.001, 0.01, 0.1, 1, 10, 100],
        'gamma': [0.001, 0.01, 0.1, 1, 10, 100],
    },
    {
        'alpha': np.logspace(0, -6, num=6, base=10),
    },
    {
        'var_smoothing': np.logspace(0, -9, num=10, base=10)
    },
]


## 学習・精度測定

In [11]:
""" trials """

trials = 10

for name, clf, param in zip(names, clfs, grid_params):
    accuracy_sum = 0.0
    print(f'[{name}]:\n\taccuracy\t[', end='')
    # start = time.perf_counter()

    for _ in range(trials):
        X_train, X_test, y_train, y_test = train_test_split(X, y)
        grid_search = GridSearchCV(clf, param)
        grid_search.fit(X_train, y_train)

        accuracy = np.mean(cross_val_score(grid_search, X_test, y_test))
        accuracy_sum += accuracy
        print(f' {accuracy:.2}', end='')

    # end = time.perf_counter()
    print(f' ]\n\tmean\t[ {accuracy_sum / trials:.3} ]')
    # print(f'time\t[ {end - start:.3} ]')


time	[ 0.485 ]
time	[ 1.26 ]
time	[ 62.3 ]
time	[ 0.316 ]


## 測定結果

測定結果は次の形式で出力している．

```
[手法]:
	accuracy [測定精度の平均値(10試行)]
	mean: [accuracyの平均値]
```



1.
	```
	[kNN]:
		accuracy [ 0.84 0.76 0.52 0.72 0.68 0.88 0.68 0.6 0.68 0.72 ]
		mean     [ 0.708 ]
	[SVM]:
		accuracy [ 0.76 0.92 0.68 0.6 0.92 0.64 0.8 0.8 0.88 0.84 ]
		mean     [ 0.784 ]
	[MLP]:
		accuracy [ 0.76 1.0 0.88 0.92 0.84 0.84 0.68 0.72 0.72 0.72 ]
		mean     [ 0.808 ]
	[NB]:
		accuracy [ 0.48 0.68 0.76 0.88 0.76 0.72 0.72 0.64 0.76 0.76 ]
		mean     [ 0.716 ]
	```
2.
	```
	[kNN]:
		accuracy [ 0.68 0.68 0.72 0.76 0.72 0.8 0.52 0.68 0.72 0.72 ]
		mean     [ 0.7 ]
	[SVM]:
		accuracy [ 0.8 0.76 0.84 0.92 0.8 0.76 0.72 0.8 0.96 0.64 ]
		mean     [ 0.8 ]
	[MLP]:
		accuracy [ 0.76 0.76 0.76 0.76 0.8 0.76 0.92 0.84 0.96 0.64 ]
		mean     [ 0.796 ]
	[NB]:
		accuracy [ 0.8 0.68 0.8 0.56 0.6 0.72 0.76 0.68 0.76 0.8 ]
		mean     [ 0.716 ]
	```
3.
	```
	[kNN]:
		accuracy [ 0.84 0.72 0.72 0.72 0.76 0.88 0.76 0.8 0.8 0.64 ]
		mean     [ 0.764 ]
	[SVM]:
		accuracy [ 0.72 0.48 0.64 0.8 0.52 0.76 0.6 0.56 0.84 0.8 ]
		mean     [ 0.672 ]
	[MLP]:
		accuracy [ 0.88 0.6 0.64 0.72 0.92 0.84 0.8 0.72 0.72 0.76 ]
		mean     [ 0.76 ]
	[NB]:
		accuracy [ 0.88 0.72 0.72 0.68 0.88 0.88 0.76 0.72 0.52 0.68 ]
		mean     [ 0.744 ]
	```
4.
	```
	[kNN]:
		accuracy [ 0.6 0.68 0.56 0.76 0.84 0.56 0.68 0.64 0.56 0.52 ]
		mean     [ 0.64 ]
	[SVM]:
		accuracy [ 0.52 0.6 0.68 0.76 0.68 0.72 0.72 0.76 0.64 0.68 ]
		mean     [ 0.676 ]
	[MLP]:
		accuracy [ 0.88 0.64 0.64 0.64 0.8 0.68 0.64 0.6 0.64 0.68 ]
		mean     [ 0.684 ]
	[NB]:
		accuracy [ 0.48 0.6 0.84 0.72 0.6 0.4 0.48 0.32 0.72 0.64 ]
		mean     [ 0.58 ]
	```
5.
	```
	[kNN]:
		accuracy [ 0.8 0.52 0.64 0.48 0.6 0.68 0.44 0.48 0.76 0.52 ]
		mean     [ 0.592 ]
	[SVM]:
		accuracy [ 0.68 0.64 0.6 0.64 0.56 0.6 0.52 0.52 0.6 0.52 ]
		mean     [ 0.588 ]
	[MLP]:
		accuracy [ 0.68 0.56 0.56 0.76 0.52 0.88 0.6 0.8 0.8 0.76 ]
		mean     [ 0.692 ]
	[NB]:
		accuracy [ 0.6 0.56 0.6 0.56 0.56 0.6 0.6 0.64 0.76 0.36 ]
		mean     [ 0.584 ]
	```

## 考察

各試行における精度の順位を次の表に示す．

|試行No.|kNN|SVM|MLP|NB|
|-|-|-|-|-|
|1|4|2|1|3|
|2|4|1|2|3|
|3|1|4|2|3|
|4|3|2|1|4|
|5|2|3|1|4|

これらの試行から，MLPの精度がNBより優ると推察される．
またその他の手法については，優劣を評価できない．
ただし，ハイパーパラメータには調整を行っていない項目も存在するため，調整するハイパーパラメータについての更なる評価が必要である．

また，上試行とは別に計測した学習の処理時間(sec)を次の表に示す．
ただし，この計測の際には学習とは無関係な処理を行っていない．

|試行No.|kNN|SVM|MLP|NB|
|-|-|-|-|-|
|1|0.489|1.25|63.2|0.308|
|2|0.508|1.28|64.9|0.314|
|3|0.486|1.31|64.6|0.304|
|4|0.497|1.26|67.4|0.354|
|5|0.485|1.26|62.3|0.316|

これらの試行から，本試行におけるハイパーパラメータの調整を行った場合，学習に要する時間はMLP > SVM > kNN > NBである．
このうちMLPの学習に要する時間は他の3手法と比較して10倍以上と大きく，MLPは学習に大きな時間を要すると言える．
